In [27]:
import numpy as np
import pandas as pd

In [28]:
df = pd.read_csv('diabetes.csv')

In [29]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [30]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [31]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [32]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [33]:
X.shape

(768, 8)

In [48]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [35]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))

model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_test,y_test))

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.3904 - loss: 0.7820 - val_accuracy: 0.4740 - val_loss: 0.7133
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5193 - loss: 0.7049 - val_accuracy: 0.6667 - val_loss: 0.6428
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6715 - loss: 0.6413 - val_accuracy: 0.7240 - val_loss: 0.5945
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7396 - loss: 0.5895 - val_accuracy: 0.7500 - val_loss: 0.5589
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7522 - loss: 0.5598 - val_accuracy: 0.7708 - val_loss: 0.5349
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7564 - loss: 0.5480 - val_accuracy: 0.7656 - val_loss: 0.5162
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7613 - loss: 0.5132 - val_accuracy: 0.7656 - val_loss: 0.5017
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7705 - loss: 0.5142 - val_accuracy: 0.7760 - val_loss

In [37]:
#1. how to select appropriate optimizer
#2. no. of nodes in a layer
#3. no. of layers
#4. all in one model

In [38]:
pip install -U keras-tuner

In [39]:
import keras_tuner as kt

In [50]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

    if counter == 0:
      model.add(
          Dense(
              hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
              activation= hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']),
              input_dim=X_train.shape[1]
              )
          )
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5])))

    else:

      model.add(
          Dense(
              hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
              activation= hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])
              )
          )

      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5])))
    counter += 1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer= hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd','nadam','adadelta']),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [51]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    directory='mydir',
    project_name='BestModelFinder'
)

Reloading Tuner from mydir/BestModelFinder/tuner0.json


In [52]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [53]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 120,
 'activation0': 'tanh',
 'optimizer': 'rmsprop',
 'units1': 8,
 'activation1': 'relu',
 'units2': 8,
 'activation2': 'relu',
 'units3': 8,
 'activation3': 'relu',
 'units4': 8,
 'activation4': 'relu'}

In [54]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [55]:
model.fit(X_train, y_train, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.7073 - loss: 0.6567 - val_accuracy: 0.8021 - val_loss: 0.6202
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7365 - loss: 0.6268 - val_accuracy: 0.8021 - val_loss: 0.6013
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7797 - loss: 0.5878 - val_accuracy: 0.8021 - val_loss: 0.5747
Epoch 10/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7402 - loss: 0.5988 - val_accuracy: 0.7917 - val_loss: 0.5544
Epoch 11/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7602 - loss: 0.5759 - val_accuracy: 0.8125 - val_loss: 0.5231
Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7492 - loss: 0.5584 - val_accuracy: 0.7969 - val_loss: 0.5040
Epoch 13/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7789 - loss: 0.5194 - val_accuracy: 0.7865 - val_loss: 0.4880
Epoch 14/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7395 - loss: 0.5530 - val_accuracy: 0.78